In [ ]:
import awkward
import numpy as np
from coffea import nanoevents

def get_array(array):
    return array.compute() if nanoevents_mode == "dask" else array

nanoevents_mode = "virtual"

access_log = []
events = nanoevents\
         .NanoEventsFactory\
         .from_root({"coffea/tests/samples/nano_dy.root": "Events"}, mode=nanoevents_mode, access_log=access_log)\
         .events()

muons = events.Muon
jets = events.Jet
met = events.MET

In [ ]:
def some_event_weight(ones):
    return (1.0 + np.array([0.05, -0.05], dtype=np.float32)) * ones[:, None]

events.add_systematic("RenFactScale", "UpDownSystematic", "weight", some_event_weight)
events.add_systematic("XSectionUncertainty", "UpDownSystematic", "weight", some_event_weight)

def muon_pt_scale(pt):   
    return (1.0 + np.array([0.05, -0.05], dtype=np.float32)) * pt[:, None]

def muon_pt_resolution(pt):   
    return np.random.normal(pt[:,None], np.array([0.02, 0.01], dtype=np.float32))

def muon_eff_weight(ones):
    return (1.0 + np.array([0.05, -0.05], dtype=np.float32)) * ones[:, None]

def muon_pt_phi_systematic(ptphi):
    pt_var = (1.0 + np.array([0.05, -0.05], dtype=np.float32)) * ptphi.pt[:, None]
    phi_var = (1.0 + np.array([0.1, -0.1], dtype=np.float32)) * ptphi.phi[:, None]
    return awkward.zip({"pt": pt_var, "phi": phi_var}, depth_limit=1)

muons.add_systematic("PtScale", "UpDownSystematic", "pt", muon_pt_scale)
muons.add_systematic("PtResolution", "UpDownSystematic", "pt", muon_pt_resolution)
muons.add_systematic("EfficiencySF", "UpDownSystematic", "weight", muon_eff_weight)
muons.add_systematic("PtPhiSystematic", "UpDownMultiSystematic", ("pt", "phi"), muon_pt_phi_systematic)

def jet_pt_scale(pt):   
    return (1.0 + np.array([0.10, -0.10], dtype=np.float32)) * pt[:, None]

def jet_pt_resolution(pt):   
    return np.random.normal(pt[:,None], np.array([0.20, 0.10], dtype=np.float32))

def jet_pt_phi_systematic(ptphi):
    pt_var = (1.0 + np.array([0.10, -0.10], dtype=np.float32)) * ptphi.pt[:, None]
    phi_var = (1.0 + np.array([0.2, -0.2], dtype=np.float32)) * ptphi.phi[:, None]
    return awkward.zip({"pt": pt_var, "phi": phi_var}, depth_limit=1)

jets.add_systematic("PtScale", "UpDownSystematic", "pt", jet_pt_scale)
jets.add_systematic("PtResolution", "UpDownSystematic", "pt", jet_pt_resolution)
jets.add_systematic("PtPhiSystematic", "UpDownMultiSystematic", ("pt", "phi"), jet_pt_phi_systematic)

def met_pt_scale(pt):
    return (1.0 + np.array([0.03, -0.03], dtype=np.float32)) * pt[:, None]

def met_pt_phi_systematic(ptphi):
    pt_var = (1.0 + np.array([0.03, -0.03], dtype=np.float32)) * ptphi.pt[:, None]
    phi_var = (1.0 + np.array([0.05, -0.05], dtype=np.float32)) * ptphi.phi[:, None]
    return awkward.zip({"pt": pt_var, "phi": phi_var}, depth_limit=1)

met.add_systematic("PtScale", "UpDownMultiSystematic", "pt", met_pt_scale)
met.add_systematic("PtPhiSystematic", "UpDownMultiSystematic", ("pt", "phi"), met_pt_phi_systematic)

In [ ]:
renfact_up = events.systematics.RenFactScale.up.weight_RenFactScale
get_array(renfact_up)

In [ ]:
muon_pt = awkward.flatten(muons.pt)
get_array(muon_pt)

In [ ]:
muon_PtScale_up = awkward.flatten(muons.systematics.PtScale.up)
get_array(muon_PtScale_up)

In [ ]:
muon_PtScale_up_pt = awkward.flatten(muons.systematics.PtScale.up.pt)
get_array(muon_PtScale_up_pt)

In [ ]:
muons_PtPhiSystematic_up = awkward.flatten(muons.systematics.PtPhiSystematic.up)
get_array(muons_PtPhiSystematic_up)

In [ ]:
muons_PtPhiSystematic_up_pt = awkward.flatten(muons.systematics.PtPhiSystematic.up.pt)
get_array(muons_PtPhiSystematic_up_pt)

In [ ]:
muons_PtPhiSystematic_up_phi = awkward.flatten(muons.systematics.PtPhiSystematic.up.phi)
get_array(muons_PtPhiSystematic_up_phi)

In [ ]:
jets_pt = awkward.flatten(jets.pt)
get_array(jets_pt)

In [ ]:
jets_PtScale_up = awkward.flatten(jets.systematics.PtScale.up)
get_array(jets_PtScale_up)

In [ ]:
jets_PtScale_up_pt = awkward.flatten(jets.systematics.PtScale.up.pt)
get_array(jets_PtScale_up_pt)

In [ ]:
jets_PtPhiSystematic_up = awkward.flatten(jets.systematics.PtPhiSystematic.up)
get_array(jets_PtPhiSystematic_up)

In [ ]:
jets_PtPhiSystematic_up_pt = awkward.flatten(jets.systematics.PtPhiSystematic.up.pt)
get_array(jets_PtPhiSystematic_up_pt)

In [ ]:
jets_PtPhiSystematic_up_phi = awkward.flatten(jets.systematics.PtPhiSystematic.up.phi)
get_array(jets_PtPhiSystematic_up_phi)

In [ ]:
met_pt = met.pt
get_array(met_pt)

In [ ]:
met_PtScale_up = met.systematics.PtScale.up
get_array(met_PtScale_up)

In [ ]:
met_PtScale_up_pt = met.systematics.PtScale.up.pt
get_array(met_PtScale_up_pt)

In [ ]:
met_PtPhiSystematic_up = met.systematics.PtPhiSystematic.up
get_array(met_PtPhiSystematic_up)

In [ ]:
met_PtPhiSystematic_up_pt = met.systematics.PtPhiSystematic.up.pt
get_array(met_PtPhiSystematic_up_pt)

In [ ]:
met_PtPhiSystematic_up_phi = met.systematics.PtPhiSystematic.up.phi
get_array(met_PtPhiSystematic_up_phi)

In [ ]:
# TODO: Make it so that syst_muons.Y > X returns boolean values
#       for all variations over Y. 
#       Requires some tracking of (pieces of) "what".